# Homework 1 - Streaming (10 pts)

*Unisse Chua (uc288)*

### DUE: 06/26/2018 before class at 2:00pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_Streaming_NetID.ipynb.

## Task 1 (5 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [1]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

hiredCabs = 0
maxHiredCabs = 0
for row in csvRows('taxi_events.csv'):
    if row['event']=='pickup':
        hiredCabs += 1
        if maxHiredCabs<hiredCabs:
            maxHiredCabs = hiredCabs
    else:
        hiredCabs -= 1
    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

`hiredCabs` holds the number of cabs currently hired. For every pickup event, the counter incrememts and for every dropoff, the counter decrements. To get the maximum number of cabs hired at the given moment, once a pickup event is logged, the `maxHiredCabs` value should also be updated to keep track of the current largest value. 

## Task 2 (5 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [5]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
dropoffTimes = []
for row in csvRows('taxi_trips.csv'):
    dropoffTimes = [t for t in dropoffTimes if t > row['pickup_time']]
    dropoffTimes.append(row['dropoff_time'])
    
    maxHiredCabs = max(maxHiredCabs, len(dropoffTimes))
    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

To keep track if a cab is still hired, we keep the dropoff times in a list and count how many are still being used. However, we need to remove the cabs from the dropoff time list once the dropoff time has passed the current pickup time.

At each time, we count how many cabs are hired and store the highest value to `maxHiredCabs` to know the maximum number of cabs at any point in time.